In [56]:
# Import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
df = pd.read_csv("D:/Datasets/Churn_Modelling.csv")

# Veri keşfi

In [58]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [59]:
len(df.columns)

14

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [61]:
df.groupby(['Exited']).RowNumber.agg(['count'])

,count
Exited,
0,7963
1,2037


In [62]:
# Bağımlı ve bağımsız değişkenleri ayırma
# RowNumber, CustomerId, Surname hedef değişkeni tahmin etme bakımından işe yaramaz. Bu yüzden 0,1,2 indeksli nitelikleri
# Nitelikler matrisine dahil etmiyoruz. Yukarıda kaç tane nitelik olduğunu yazdırmıştık. Şimdi 0,1,2 hari ise 3'ten başlayacak. 
# Python'da indeks değeri 0'dan başladığına ve 14 nitalik olduğuna göre son niteliğin indeks değeri 13 olacaktır. iloc ile sütun
#seçerken ilk değer dahil ikinci değer hariçtir. bu sebeple 3'ü dahil 13'ü hariç tutacağız. Çünkü son indeks değeri Exited yani
# hedef değişkene geliyor. İlk : tüm satırlar "," sonrası sütunlardan seçilecek filtre. 3:13 demek 3 dahil 13 hariç indeksli
# sütunları seç demek

X = df.iloc[:,3:13].values

# Hedef niteliğimiz 13. indekste idi. O sebeple : ile tüm satırları "," den sonra 13 ile hedef niteliğin indeksini seçiyoruz.
y = df.iloc[:,13].values

In [63]:
X.shape

(10000, 10)

In [64]:
y.shape

(10000,)

# Kategorik Nitelikler için LabelEncoder

In [65]:
# Hedef nitelik 0 ve 1 olmak üzere zaten nümerik değer olduğu için onu LabelEncoder ile kodlamaya gerek yok.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# X nitelikler matrisini incelediğimizde 1'inci indekste Geography 2'nci indekste Gender'ın kategorik olduğunu görüyoruz.
# Önce Geagraphy için LabelEncoder yaratalım
labelEncoder_Geo = LabelEncoder()
X[:, 1] = labelEncoder_Geo.fit_transform(X[:, 1])

In [66]:
# Şimdi Gender için
labelEncoder_Gender = LabelEncoder()
X[:,2] = labelEncoder_Gender.fit_transform(X[:,2])

# OneHotEncoder

In [67]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [68]:
# Birden sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

# one_hot_encode_with_ct bizim kendi verdiğiiz isimdir. onehotencoder'ı yukarıda yarattık. [1] ise dönüşecek
# niteliklerin indeks değerleridir (Geography) Gender binary olduğu için sokmaya gerek yok.
ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [1]),], 
    remainder='passthrough'
)

In [69]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [70]:
X.shape

(10000, 12)

In [71]:
X[:3]

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]],
      dtype=object)

In [72]:
# Gölge Değişken Tuzağından kurtulmak için Yeni oluşturulan gölge değişkenlerden birini çıkaralım (0 çıkıyor)
X = X[:,1:]

In [73]:
X.shape

(10000, 11)

# Feature Scaling

In [74]:
# Nitelikler matrisini (X) StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [75]:
X[:3]

array([[-0.57873591, -0.57380915, -0.32622142, -1.09598752,  0.29351742,
        -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
         0.02188649],
       [-0.57873591,  1.74273971, -0.44003595, -1.09598752,  0.19816383,
        -1.38753759,  0.11735002, -0.91158349, -1.54776799,  0.97024255,
         0.21653375],
       [-0.57873591, -0.57380915, -1.53679418, -1.09598752,  0.29351742,
         1.03290776,  1.33305335,  2.52705662,  0.64609167, -1.03067011,
         0.2406869 ]])

# Eğitim ve Test Seti olarak Ayırmak

In [76]:
from sklearn.model_selection import train_test_split

#Sıralamaya dikkat. Aynı sonuçlar için random_state değeri aynı olmalıdır.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=142)

# Bölüm 2 ANN

In [77]:
# Bu bölümden önce aşağıdaki kütüphaneler kurulmalıdır.
# !pip install theano

# !pip install tensorflow

# !pip install keras


In [78]:
# Keras kütüphanelerini indirme
import keras
from keras.models import Sequential
from keras.layers import Dense

## YAPAY SİNİR AĞI NESNESİ OLUŞTURMA

In [79]:
# Yapay sinir ağı iki farklı şekilde başlatılabilir: 
# 1. Katmanlar dizilimi (Sequantial layers) olarak. 
# 2. Graph olarak.
# Birinci yolda sadece yukarıda indirilen Sequantial sınıfından nesne yaratılır. 
# Bu da Karar ağacı, SVM gibi bir sınıflandırıcı olacak.
classifier = Sequential()

## Bir Sinir Ağını Stochastic Gradient Descent ile Eğitmenin Aşamaları

    Sıfıra yakın rastgele değerler ile ağırlık katsayılarını belirle.
    Input layera ilk gözlemi(bir satır) her bir düğüme bir nitelik (nitelikler matrisi sütün sayısı kadar) düşecek şekilde ver.
    Ağı çalıştır ve ilk tahmin y değerini üret.
    Üretilen y ile tahmin edilen y arasındaki hatayı hesapla.
    Hesaplanan hatayı geri besleme olarak gönder ve her bir ağırlık katsayısını güncelle.
    1 ve 5 arasını ya her bir satırdan sonra veya belli bir satır sayısından (grup-batch) sonra tekrarla.
    Veri setindeki tüm satırların bitmesi bir epoch oluşturur. Bunu defalarca tekrarla.

## GİRDİ VE İLK GİZLİ KATMANI EKLEME

In [80]:
# classifier nesnesinin add() metodu içine Dense() Sınıfının nesnesnesini parametre olarak veriyoruz.
# Dense içindeki ilk parametre 
# kernel_initializer (eski init): Başlangıç ağırlıklarını belirler. uniform fonksiyonunu kullanıyoruz.
# input_dim: Yapay sinir ağı nesnesine ilk katman ekleme işlemi yapılırken mutlaka girdi katmanında kaç düğüm olacağı bildirilir.
# aksi halde ilk gizli katman kendisine kaç düğümden bağlantı olacak bilemez. 
# Bu rakam nitelikler matrisindeki sütun sayısıdır yani 11
# units (eski:output_dim): Bu sayının nasıl verileceğine dair sağlam bir kural yok. Biraz sanatkarlık ve tecrübe
# gerektiriyor. Tecrübeye dayalı ipucu: Girdi ve çıktı düğüm sayısının ortalamasını verebilirsin. 
# Bu örnekte (11+1=12/2=6) Ya da parametre tuning tekniklerini kullanabilirsin.
# activation: Gizli düğümler için rectifier çıktı düğüm için sigmoid olsun. Rectifier action parametresinde relu olarak tanımlı


classifier.add(Dense(kernel_initializer = 'uniform', input_dim = 11, units = 6,  activation = 'relu'))

## İKİNCİ GİZLİ KATMANI EKLEME

In [81]:
# İkinci Gizli Katmanı Eklemek

# Yukarıda ilk gizli katman ve düğüm sayısı belli olduğu için burada input_dim parametresi kullanmaya gerek yok.
# Gizli katman sayısını (units) aynı tutuyoruz. kaba formül kullandık yine.
classifier.add(Dense(kernel_initializer = 'uniform', units = 4,  activation = 'relu'))

## ÇIKTI KATMANI (OUTPUT LAYER) EKLEME

In [82]:
# Bu katmanın bir öncekinden farkı düğüm sayısı ve activasyon fonksiyonu olacak

classifier.add(Dense(kernel_initializer = 'uniform', units = 1,  activation = 'sigmoid'))

## YAPAY SINIR AĞINI DERLEME (COMPILE ANN)

In [83]:
# optimizer: Stochastic Gradient Descent'i temsilen adam
# loss: SGD'ni optimizasyonu için kullanılacak loss function. 
#       Tahmin y ile gerçek ye değeri arasını hesaplayıp en optimal değeri
# SGD'ye buldurur. Logaritmik loss function. Binary olduğu için binary_crossentropy kullanıyoruz.
# metrics: İlave olarak burada model değerlendirme kriterleri belirlenir. Bir liste halinde verilir. 
#          Biz şimdilik sadece accuracy veriyoruz.


classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Yapay Sinir Ağını Eğitim Verisi ile Eğitmek

In [84]:
classifier.fit(X_train, y_train, batch_size=5, epochs=200)

Epoch 1/200
8000/8000 [==============================] - 1s 146us/step - loss: 0.4707 - acc: 0.7916
Epoch 2/200
8000/8000 [==============================] - 1s 119us/step - loss: 0.4298 - acc: 0.7916
Epoch 3/200
8000/8000 [==============================] - 1s 121us/step - loss: 0.4237 - acc: 0.8136
Epoch 4/200
8000/8000 [==============================] - 1s 120us/step - loss: 0.4196 - acc: 0.8251
Epoch 5/200
8000/8000 [==============================] - 1s 125us/step - loss: 0.4174 - acc: 0.8273
Epoch 6/200
8000/8000 [==============================] - 1s 121us/step - loss: 0.4156 - acc: 0.8313
Epoch 7/200
8000/8000 [==============================] - 1s 119us/step - loss: 0.4141 - acc: 0.8294
Epoch 8/200
8000/8000 [==============================] - 1s 119us/step - loss: 0.4128 - acc: 0.8328
Epoch 9/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.4116 - acc: 0.8310
Epoch 10/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.4106 - acc: 0.8309

8000/8000 [==============================] - 1s 110us/step - loss: 0.3465 - acc: 0.8571
Epoch 80/200
8000/8000 [==============================] - 1s 110us/step - loss: 0.3468 - acc: 0.8564
Epoch 81/200
8000/8000 [==============================] - 1s 111us/step - loss: 0.3467 - acc: 0.8594
Epoch 82/200
8000/8000 [==============================] - 1s 110us/step - loss: 0.3464 - acc: 0.8591
Epoch 83/200
8000/8000 [==============================] - 1s 111us/step - loss: 0.3459 - acc: 0.8590
Epoch 84/200
8000/8000 [==============================] - 1s 115us/step - loss: 0.3457 - acc: 0.8609
Epoch 85/200
8000/8000 [==============================] - 1s 114us/step - loss: 0.3461 - acc: 0.8595
Epoch 86/200
8000/8000 [==============================] - 1s 109us/step - loss: 0.3448 - acc: 0.8588
Epoch 87/200
8000/8000 [==============================] - 1s 110us/step - loss: 0.3454 - acc: 0.8604
Epoch 88/200
8000/8000 [==============================] - 1s 113us/step - loss: 0.3449 - acc: 0.8605
Epo

8000/8000 [==============================] - 1s 112us/step - loss: 0.3421 - acc: 0.8583
Epoch 160/200
8000/8000 [==============================] - 1s 112us/step - loss: 0.3425 - acc: 0.8600
Epoch 161/200
8000/8000 [==============================] - 1s 113us/step - loss: 0.3426 - acc: 0.8576
Epoch 162/200
8000/8000 [==============================] - 1s 121us/step - loss: 0.3416 - acc: 0.8598
Epoch 163/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.3419 - acc: 0.8606
Epoch 164/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.3424 - acc: 0.8593
Epoch 165/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.3420 - acc: 0.8610
Epoch 166/200
8000/8000 [==============================] - 1s 117us/step - loss: 0.3421 - acc: 0.8599
Epoch 167/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.3419 - acc: 0.8586
Epoch 168/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.3416 - acc: 0

# Yapay Sinir Ağını Test verisi ile Test Etmek

In [85]:
y_pred = classifier.predict(X_test)

In [86]:
y_pred[:4]

array([[0.35882223],
       [0.90301347],
       [0.29493663],
       [0.03977633]], dtype=float32)

In [88]:
# y_pred için olasılık değerlerini görüyoruz. 0.5'ten büyük olanları 1 olarak kabul edelim.
y_pred = [1 if i > 0.5 else 0 for i in y_pred]

In [89]:
y_pred[:4]

[0, 1, 0, 0]

In [90]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1577,   53],
       [ 203,  167]], dtype=int64)

In [91]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy_score(y_pred=y_pred, y_true=y_test)

0.872